##### 问题:
我们编写了许多类，把它们当做数据结构来用。但是我们厌倦了编写高度重复且样式
相同的__init__()函数。

##### 解决方案:
通常我们可以将初始化数据结构的步骤归纳到一个单独的__init__()函数中，并将其定
义在一个公共的基类中。示例如下：

In [3]:
import math
class Structure:
    # Class variable that specifies expected fields
    _fields= []
    def __init__(self, *args):
        # len(args)指的是列表中元素的个数
        if len(args) != len(self._fields): 
            raise TypeError('Expected {} arguments'.format(len(self._fields)))
        # Set the arguments
        for name, value in zip(self._fields, args):
            setattr(self, name, value)
# Example class definitions
if __name__ == '__main__':
    class Stock(Structure):                     #类的继承
        _fields = ['name', 'shares', 'price']   #重写类内的属性
    class Point(Structure):
        _fields = ['x','y']
    class Circle(Structure):
        _fields = ['radius']
        def area(self):
            return math.pi * self.radius ** 2


如果使用这些类，就会发现它们非常易于构建。示例如下：

In [2]:
s = Stock('ACME', 50, 91.1)
p = Point(2, 3)
c = Circle(4.5)
# s2 = Stock('ACME', 50)

我们应该提供对关键字参数的支持，这里有几种设计上的选择。一种选择就是对关键
字参数做映射，这样它们就只对应于定义在_fields 中的属性名。示例如下：

In [4]:
class Structure:
    _fields= []
    def __init__(self, *args, **kwargs):
        if len(args) > len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))
        # Set all of the positional arguments
        for name, value in zip(self._fields, args):  #把属性名和对应的值顺序打包
            setattr(self, name, value) #设置属性并设置属性名
        # Set the remaining keyword arguments
        for name in self._fields[len(args):]: #对字典匹配,只匹配字典类型的。self._fields[len(args):]指的是fields中字典类型的数据
            setattr(self, name, kwargs.pop(name))
        # Check for any remaining unknown arguments
        if kwargs:
             raise TypeError('Invalid argument(s): {}'.format(','.join(kwargs)))
# Example use
if __name__ == '__main__':
    class Stock(Structure):
        _fields = ['name', 'shares', 'price'] 
    s1 = Stock('ACME', 50, 91.1) #必须顺序输入
    s2 = Stock('ACME', 50, price=91.1)
    s3 = Stock('ACME', price=91.1, shares=50) #字典传入数据

另一种可能的选择是利用关键字参数来给类添加额外的属性，这些额外的属性是没有
定义在_fields 中的。示例如下：

<b>(上下原理相似)</b>

In [6]:
class Structure:
    # Class variable that specifies expected fields
    _fields= []
    def __init__(self, *args, **kwargs):
        if len(args) != len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))
        # Set the arguments
        for name, value in zip(self._fields, args):
            setattr(self, name, value)
        # Set the additional arguments (if any)
        extra_args = kwargs.keys() - self._fields
        for name in extra_args:
            setattr(self, name, kwargs.pop(name))
        if kwargs:
            raise TypeError('Duplicate values for {}'.format(','.join(kwargs)))
# Example use
if __name__ == '__main__':
    class Stock(Structure):
        _fields = ['name', 'shares', 'price']
        s1 = Stock('ACME', 50, 91.1)
        s2 = Stock('ACME', 50, price=91.1)
        # s3 = Stock('ACME', 50, 91.1, date='8/2/2012')

如果要编写的程序中有大量小型的数据结构，那么定义一个通用型的__init__()方法会
特别有用。相比于手动编写每个__init__()方法，这么做可使得代码量大大
减少：

我们给出的实现中，一个微妙之处在于使用了 setattr()函数来设定属性值。与之相反的
是，有人可能会倾向于直接访问实例字典。示例如下：

In [ ]:
class Structure:
    #Class variable that specifies expected fields
    _fields= []
    def __init__(self, *args):
        if len(args) != len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))
        # Set the arguments (alternate)
        self.__dict__.update(zip(self._fields,args))

尽管这么做行得通，但像这样假设子类的实现通常是不安全的。如果某个子类决定使
用__slots__或者用 property（也可以是描述符）包装了某个特定的属性，直接访问实例
字典就会产生崩溃。我们给出的解决方案已经尽可能地做到通用，不会对子类的实现
做任何假设。

这种技术的一个潜在缺点就是会影响到 IDE（集成开发环境）的文档和帮助功能。如
果用户针对某个特定的类寻求帮助，那么所需的参数将不会以正常的形式来表述。示
例如下：

In [7]:
help(Stock) 


Help on class Stock in module __main__:

class Stock(Structure)
 |  Stock(*args, **kwargs)
 |  
 |  Method resolution order:
 |      Stock
 |      Structure
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  _fields = ['name', 'shares', 'price']
 |  
 |  s1 = <__main__.Stock object>
 |  
 |  s2 = <__main__.Stock object>
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Structure:
 |  
 |  __init__(self, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from Structure:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



这些问题可以通过在__init__()函数中强制施行类型签名来解决，相关内容请参阅 9.16 节。

应该指出的是，也可以采用所谓的“frame hack”技巧来实现自动化的实例变量初始化
处理，只要编写一个功能函数即可。示例如下：

In [ ]:
def init_fromlocals(self):
    import sys
    locs = sys._getframe(1).f_locals
    for k, v in locs.items():
        if k != 'self':
            setattr(self, k, v)
class Stock:
    def __init__(self, name, shares, price):
        init_fromlocals(self)

在这种方法中，函数 init_fromlocals()利用 sys.\_getframe()来获取调用方的局部变量。如
果在__init__()方法中首先调用这个函数，那么获取到的局部变量就和传递给__init__()
方法的参数是一致的，可以轻松用来设定属性。尽管这种方法可以避免在 IDE 中出现
获取到不一致的调用签名问题，但比起解决方案中提供的方法要慢上 50%，也需要程
序员输入更多的代码，这种方法在幕后也做了更加复杂的操作。如果我们的代码不需
要这种额外的能力，那么通常更简单的方案会更好。